In [ ]:
import pandas as pd 
import numpy as np 
from catboost import CatBoostClassifier,Pool,cv,utils

# import sys
# sys.path.append('C:\\Users\peter\\Documents\\MyProjects\\PyProj\\Trav\\spel')
# import V75_scraping as vs

In [ ]:
dforg = pd.read_csv('..//all_data.csv')     
print(dforg.columns)

In [ ]:
### Features som inte används vid träning
def remove_features(df,remove_mer=[]):
    #remove_mer=['h5_perf','h5_auto','h4_perf','h4_auto', 'h3_perf', 'h2_perf']
    df.drop(['startnr','vodds','podds','bins','h1_dat','h2_dat','h3_dat','h4_dat','h5_dat'],axis=1,inplace=True) #
    if remove_mer:
        df.drop(remove_mer,axis=1,inplace=True)
    
    # df=check_unique(df.copy())
    # df=check_corr(df.copy())
    return df

In [ ]:
 ## byt ut alla NaN till text för cat_features
def replace_NaN(X_train,X_test=None, cat_features=[]):
    # print('cat_features',cat_features)
    X_train[cat_features]=X_train[cat_features].fillna('missing')
    if X_test is not None:  ## om X_test är med
        X_test[cat_features]=X_test[cat_features].fillnal('missing')    ### byt ut None-värden till texten 'None

    return X_train,X_test

In [ ]:
# create basic and clean X_train, X_test etc that can be used as is -- 
def basic_data(df, frac=0.25):
    X_train,X_test,y_train,y_test = None,None,None,None
    dfnew = remove_features(df.copy())
    dfnew['plac'] = (dfnew.plac==1) * 1
    cat_features = list(dfnew.loc[:,df.dtypes=='O'].columns)
    dfnew,_ = replace_NaN(dfnew.copy(), cat_features=cat_features)    
    
    alla_datum = df.datum.unique()
    split_dat = alla_datum[int(len(alla_datum)* (1 - 0.25))]     # större än split_dat är test

    X_train = dfnew.loc[dfnew.datum <= split_dat].copy()
    y_train=X_train.plac
    X_train.drop('plac',axis=1,inplace=True)
    
    X_test = dfnew.loc[dfnew.datum > split_dat].copy()
    y_test=X_test.plac
    X_test.drop('plac',axis=1,inplace=True)
    
    return X_train,X_test, y_train,y_test

# split_dat = dforg.datum.unique()[int(len(dforg.datum.unique())* (1 - 0.25))]  
# dforg.loc[dforg.datum<'2021-09-18']
# X_train,X_test,y_train,y_test = basic_data(dforg.copy())

In [ ]:
# Handle ekipage (häst and kusk)
# Set a smooth mean value to the features in X_train  ##
def calc_smooth_mean(X, y, by, m=100, tot_mean=None):
    Xcopy = X.copy()
    Xcopy['plac'] = y

    # Compute the number of values and the mean of each group
    agg = Xcopy.groupby(by)['plac'].agg(['count', 'mean'])
    counts = agg['count']
    means = agg['mean']

    # Compute the "smoothed" means
    smooth = (counts * means + m * tot_mean) / (counts + m)
    
    return smooth.to_dict()


from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer


In [ ]:
# df skall innehålla datum,avd,vodds
def proba_order_score(df_, y,proba):
    kassa=200
    df = df_.copy()
    df['proba'] = proba[:,1]
    df['f'] = (df.proba*df.vodds - 1) / (df.vodds-1)  # kelly formel
    df['spela'] = df.f >0
    df['insats'] = df.spela * df.f * kassa

    df.sort_values(['datum','avd','proba'],ascending=[True,True,False],inplace=True)
    proba_order=df.groupby(['datum','avd']).proba.cumcount()

    df['prob_order']=proba_order+1
    df['y'] = y
    
    return df, df.loc[df.y==1].prob_order.mean()   # mean vann per avd
    


# Kolla olika features och dess betydelse på resultatet
## data

In [ ]:

dforg = pd.read_csv('..\\all_data.csv')     
#
X_train,X_test,y_train,y_test = basic_data(dforg.copy())
cat_features = list(X_train.loc[:,X_train.dtypes=='O'].columns)


In [ ]:
# datum, avd, ekipage (raw, kodat) - blir det skillnad med och utan 'streck' 
# Spara alla värden nedan                           proba_order         AUC                 Accuracy
# 1.  datun, avd, häst och kusk                     3.1316964285714284  0.8154899734619122  0.7407225221611429                     
# 2.  ej avd                                        3.146205357142857   0.8151640292105853  0.7406341992435652
# 3.  ej datum+avd   (bara avd borde ge sämre res)  3.1361607142857144  0.8162590757456829  0.7453960511909773 
# 4.  datum, raw_ekipage                            3.1573660714285716  0.8154180216042461  0.7378602742086335
# 5.  ej datum+avd  med raw_ekipage                 3.166294642857143   0.8143806767212379  0.737153650575579 
# 6.  datum, kodat_ekipage                          VÄNTA
# 7.  ej datum+avd, kodat_ekipage                   VÄNTA
# 8.  bästa val enligt ovan med kodad bana          VÄNTA
# 9.  gör en cv (timeSeries) med bästa val (val=3)  0.2426               0.812119            -----
     
cb = CatBoostClassifier(iterations=2000, early_stopping_rounds=100, use_best_model=True,auto_class_weights='Balanced', 
                        custom_metric=['Logloss','Accuracy','Recall','Precision','F1','AUC'], eval_metric='Accuracy', random_state=2021, verbose=False)

## De olika modellern med/utan vissa features

In [ ]:
dict_res = {'prob_score':[],'auc':[],'accuracy':[],'kommentar':[]} 

In [ ]:
# 1.    datum, avd, häst och kusk
X_train,X_test,y_train,y_test = basic_data(dforg.copy())
X_train.drop('streck',axis=1,inplace=True)
X_test.drop('streck',axis=1,inplace=True)
res=cb.fit(X_train,y_train,eval_set=(X_test,y_test),cat_features=cat_features)
logloss, auc, accuracy = res.get_best_score()['validation']['Logloss'],res.get_best_score()['validation']['AUC'],res.get_best_score()['validation']['Accuracy']

X_test[['datum','avd','vodds']] = dforg[['datum','avd','vodds']]
X,prob_score = proba_order_score(X_test,y_test,res.predict_proba(X_test))

dict_res['prob_score'].append(prob_score), dict_res['auc'].append(auc), dict_res['accuracy'].append(accuracy)
dict_res['kommentar'].append('inkl datum,avd, kusk,häst')

In [ ]:
# 2.    drop avd
X_train,X_test,y_train,y_test = basic_data(dforg.copy())

X_train.drop('streck',axis=1,inplace=True)
X_test.drop('streck',axis=1,inplace=True)
X_train.drop(['avd'],axis=1,inplace=True)
X_test.drop(['avd'],axis=1,inplace=True)
cat_features = list(X_train.loc[:,X_train.dtypes=='O'].columns)
res=cb.fit(X_train,y_train,eval_set=(X_test,y_test),cat_features=cat_features)
logloss, auc, accuracy = res.get_best_score()['validation']['Logloss'],res.get_best_score()['validation']['AUC'],res.get_best_score()['validation']['Accuracy']
X_test[['datum','avd','vodds']] = dforg[['datum','avd','vodds']]
X,prob_score = proba_order_score(X_test,y_test,res.predict_proba(X_test))
dict_res['prob_score'].append(prob_score), dict_res['auc'].append(auc), dict_res['accuracy'].append(accuracy)
dict_res['kommentar'].append('inkl datum,kusk,häst')

In [ ]:
# 3.  ej datum+avd
X_train,X_test,y_train,y_test = basic_data(dforg.copy())
X_train.drop('streck',axis=1,inplace=True)
X_test.drop('streck',axis=1,inplace=True)
X_train.drop(['datum','avd'],axis=1,inplace=True)
X_test.drop(['datum','avd'],axis=1,inplace=True)
cat_features = list(X_train.loc[:,X_train.dtypes=='O'].columns)

res=cb.fit(X_train,y_train,eval_set=(X_test,y_test),cat_features=cat_features)
logloss, auc, accuracy = res.get_best_score()['validation']['Logloss'],res.get_best_score()['validation']['AUC'],res.get_best_score()['validation']['Accuracy']
proba = res.predict_proba(X_test)
X_test[['datum','avd','vodds']] = dforg[['datum','avd','vodds']]
X,prob_score = proba_order_score(X_test,y_test, proba)
dict_res['prob_score'].append(prob_score), dict_res['auc'].append(auc), dict_res['accuracy'].append(accuracy)
dict_res['kommentar'].append('inkl kusk,häst')

In [ ]:
# 4.  datum + raw_ekipage
X_train,X_test,y_train,y_test = basic_data(dforg.copy())
X_train.drop('streck',axis=1,inplace=True)
X_test.drop('streck',axis=1,inplace=True)
X_train.drop(['avd'],axis=1,inplace=True)
X_test.drop(['avd'],axis=1,inplace=True)
X_train['ekipage'] = X_train['häst'].str.cat(X_train.kusk,sep=',')
X_train.drop(['häst','kusk'],axis=1,inplace=True) 
X_test['ekipage'] = X_test['häst'].str.cat(X_test.kusk,sep=',')
X_test.drop(['häst','kusk'],axis=1,inplace=True) 

cat_features = list(X_train.loc[:,X_train.dtypes=='O'].columns)

res=cb.fit(X_train,y_train,eval_set=(X_test,y_test),cat_features=cat_features)
logloss, auc, accuracy = res.get_best_score()['validation']['Logloss'],res.get_best_score()['validation']['AUC'],res.get_best_score()['validation']['Accuracy']
proba = res.predict_proba(X_test)
X_test[['datum','avd','vodds']] = dforg[['datum','avd','vodds']]
X,prob_score = proba_order_score(X_test,y_test,proba)
dict_res['prob_score'].append(prob_score), dict_res['auc'].append(auc), dict_res['accuracy'].append(accuracy)
dict_res['kommentar'].append('inkl datum, raw_ekipage')

In [ ]:
# 5.  ej datum+avd  med raw_ekipage
X_train,X_test,y_train,y_test = basic_data(dforg.copy())
X_train.drop('streck',axis=1,inplace=True)
X_test.drop('streck',axis=1,inplace=True)
X_train.drop(['datum','avd'],axis=1,inplace=True)
X_test.drop(['datum','avd'],axis=1,inplace=True)
X_train['ekipage'] = X_train['häst'].str.cat(X_train.kusk,sep=',')
X_train.drop(['häst','kusk'],axis=1,inplace=True) 
X_test['ekipage'] = X_test['häst'].str.cat(X_test.kusk,sep=',')
X_test.drop(['häst','kusk'],axis=1,inplace=True) 

cat_features = list(X_train.loc[:,X_train.dtypes=='O'].columns)

res=cb.fit(X_train,y_train,eval_set=(X_test,y_test),cat_features=cat_features)
logloss, auc, accuracy = res.get_best_score()['validation']['Logloss'],res.get_best_score()['validation']['AUC'],res.get_best_score()['validation']['Accuracy']
proba = res.predict_proba(X_test)
X_test[['datum','avd','vodds']] = dforg[['datum','avd','vodds']]
X,prob_score = proba_order_score(X_test,y_test, proba)
dict_res['prob_score'].append(prob_score), dict_res['auc'].append(auc), dict_res['accuracy'].append(accuracy)
dict_res['kommentar'].append('inkl raw_ekipage')

## cv
p9.  gör en cv med bästa val enl ovan  val=3 (ej datum+avd)
cv på FLAML låter sig ej göras!

In [ ]:
X_train,X_test,y_train,y_test = basic_data(dforg.copy())
X_train.drop('streck',axis=1,inplace=True)
X_test.drop('streck',axis=1,inplace=True)
X_train.drop(['datum','avd'],axis=1,inplace=True)
X_test.drop(['datum','avd'],axis=1,inplace=True)
cat_features = list(X_train.loc[:,X_train.dtypes=='O'].columns)

df = pd.concat([X_train,X_test],axis=0)
y = pd.concat([y_train,y_test])
cv_pool = Pool(df,y,cat_features=cat_features)

params = {
         'use_best_model': True,
         'eval_metric' : 'AUC',
         "loss_function": "Logloss",
         'early_stopping_rounds': 100,
         'verbose': 50,
}


In [ ]:

cv_score =cv(pool=cv_pool, 
   params=params, 
   dtrain=None, 
   iterations=2000, 
   num_boost_round=None,
   fold_count=5, 
   nfold=None,
   inverted=False,
   partition_random_seed=0,
   seed=2021, 
   shuffle=False, 
   logging_level=None, 
   stratified=True,
   as_pandas=True,
   type='TimeSeries')

In [ ]:
cv_score

In [ ]:
# cv utan streck
cv_score

In [ ]:
from IPython.display import display
print(dforg.datum.max())
display(cv_score[cv_score['test-Logloss-mean'].min() == cv_score['test-Logloss-mean']])
display(cv_score[cv_score['test-AUC-mean'].max() == cv_score['test-AUC-mean']])

## FLAML

In [ ]:
X_train,X_test,y_train,y_test = basic_data(dforg.copy())
X_train.drop('streck',axis=1,inplace=True)
X_test.drop('streck',axis=1,inplace=True)
X_train.drop(['avd'],axis=1,inplace=True)
X_test.drop(['avd'],axis=1,inplace=True)
cat_features = list(X_train.loc[:,X_train.dtypes=='O'].columns)

In [ ]:
from flaml import AutoML
automl = AutoML()
automl.fit(X_train,y_train, X_val=X_test, y_val=y_test, task='classification',  metric='roc_auc', ensemble=True, 
            n_jobs=5,split_type='time',seed=2021,early_stop=True, time_budget=240,  max_iter=2000000)

In [ ]:
# automl.best_loss, 1-automl.best_loss
proba = automl.predict_proba(X_test)
X_test[['datum','avd','vodds']] = dforg[['datum','avd','vodds']]

X,prob_score = proba_order_score(X_test,y_test,proba)
dict_res['prob_score'].append(prob_score), dict_res['auc'].append(1-automl.best_loss), dict_res['accuracy'].append(None)
dict_res['kommentar'].append('FLML datum,kusk,häst')

In [ ]:
df_res = pd.DataFrame.from_dict(dict_res)

df_res.sort_values(by='prob_score')

In [ ]:
import pickle
filename = 'feature_selection.pkl'
pickle.dump(automl, open(filename, 'wb'))

df_res.to_csv('df_res.csv',index=False)